## Data Collection

### Get US DOT Flight Delay Dataset

In [ ]:
import kagglehub
path = kagglehub.dataset_download("usdot/flight-delays")

In [ ]:
import pandas as pd

airlines_df = pd.read_csv(path + "/airlines.csv", low_memory=False, encoding='UTF-8')
airports_df = pd.read_csv(path + "/airports.csv", low_memory=False, encoding='UTF-8')
flights_df = pd.read_csv(path + "/flights.csv", low_memory=False, encoding='UTF-8')

### Get NOAA Weather Station Data

In [ ]:
import requests
import gzip
import shutil

# URL of the file
url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.csv"
file = "ghcnd-stations.csv"

# Download the file
response = requests.get(url, stream=True)
with open(file, "wb") as f:
    shutil.copyfileobj(response.raw, f)

In [ ]:
'''
FORMAT OF "ghcnd-stations.txt"

------------------------------
Variable   Columns   Type
------------------------------
ID            1-11   Character
LATITUDE     13-20   Real
LONGITUDE    22-30   Real
ELEVATION    32-37   Real
STATE        39-40   Character
NAME         42-71   Character
GSN FLAG     73-75   Character
HCN/CRN FLAG 77-79   Character
WMO ID       81-85   Character
------------------------------
'''
ghcnd_stations_df = pd.read_csv(
    file,
    header=None,
    names=["ID", "LATITUDE", "LONGITUDE", "ELEVATION", "STATE", "NAME", 
           "GSN_FLAG", "HCN_CRN_FLAG", "WMO_ID"],
    usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8],  # Select only the columns we need
    dtype={"ID": str},
    skipinitialspace=True
)
ghcnd_stations_df.head()

### Add in NOAA Weather Station IDs for airports

I have a DataFrame, `ghcnd_stations_df` that has a list of weather data stations with their lat & long stored as columns (`LATITUDE`, `LONGITUDE`). I want to use the `airports_df` which also has `LATITUDE` and `LONGITUDE` columns to find the nearest weather station to each airport. Append the `ghcnd_stations_df` `ID` column to `airports_df` as `NOAA_STATION_ID`.

In [ ]:
import numpy as np
from sklearn.neighbors import BallTree

# Remove rows with NaN lat/lon in either DataFrame
airports_df = airports_df.dropna(subset=['LATITUDE', 'LONGITUDE'])
ghcnd_stations_df = ghcnd_stations_df.dropna(subset=['LATITUDE', 'LONGITUDE'])

# Convert airport/station lat-lon to radians
airports_df['lat_rad'] = np.radians(airports_df['LATITUDE'])
airports_df['lon_rad'] = np.radians(airports_df['LONGITUDE'])
ghcnd_stations_df['lat_rad'] = np.radians(ghcnd_stations_df['LATITUDE'])
ghcnd_stations_df['lon_rad'] = np.radians(ghcnd_stations_df['LONGITUDE'])

# Build the tree from stations
stations_coords = ghcnd_stations_df[['lat_rad','lon_rad']].to_numpy()
tree = BallTree(stations_coords, metric='haversine')

# Query nearest station for each airport
airports_coords = airports_df[['lat_rad','lon_rad']].to_numpy()
distances, indices = tree.query(airports_coords, k=1)

# Append the station ID to airports_df
airports_df['NOAA_STATION_ID'] = ghcnd_stations_df.iloc[indices.flatten()]['ID'].values

# Remove the temporary columns used for calculations
airports_df.drop(columns=['lat_rad', 'lon_rad'], inplace=True)

In [ ]:
airports_df.head()

## Data Cleanup

Merges the 3 DataFrames together, first by removing their file name prefixes from the column names. Then, by adding in airport information for both the origin and destination airport (merges the Airports DF twice technically), and then joins the airlines table with the flights table. Lastly, we create a datetime object from the existing date columns

In [ ]:
# Remove file name prefix from column names
airlines_df.columns = airlines_df.columns.str.replace(r'^airlines\.csv/', '', regex=True)
airlines_df.rename(columns={'AIRLINE': 'AIRLINE NAME'}, inplace=True)

airports_df.columns = airports_df.columns.str.replace(r'^airports\.csv/', '', regex=True)

flights_df.columns = flights_df.columns.str.replace(r'^flights\.csv/', '', regex=True)

In [ ]:
# Join airlines data to flights table
flights_df = flights_df.join(airlines_df.set_index('IATA_CODE'), on='AIRLINE')

In [ ]:
# Join airports data to flights table
origin_airports = airports_df.add_prefix('origin_airport/')
destination_airports = airports_df.add_prefix('destination_airport/')

flights_df = flights_df.join(origin_airports.set_index('origin_airport/IATA_CODE'), on='ORIGIN_AIRPORT')
flights_df = flights_df.join(destination_airports.set_index('destination_airport/IATA_CODE'), on='DESTINATION_AIRPORT')

In [ ]:
# Create DATE column from YEAR, MONTH, DAY, and SCHEDULED_DEPARTURE columns
flights_df['DATE'] = pd.to_datetime(
    flights_df['YEAR'].astype(str) + '-' +
    flights_df['MONTH'].astype(str) + '-' +
    flights_df['DAY'].astype(str) + ' ' +
    flights_df['SCHEDULED_DEPARTURE'].astype(str).str.zfill(4),
    format='%Y-%m-%d %H%M'
)

In [ ]:
# Replace the values in the CANCELLATION_REASON column
flights_df['CANCELLATION_REASON'] = flights_df['CANCELLATION_REASON'].replace({
    'A': 'Airline/Carrier',
    'B': 'Weather',
    'C': 'National Air System',
    'D': 'Security'
})

### Get Weather Data

In [ ]:
import requests
import gzip
import shutil

# URL of the file
url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/2015.csv.gz"
output_gz_file = "2015.csv.gz"
output_csv_file = "2015.csv"

# Download the file
response = requests.get(url, stream=True)
with open(output_gz_file, "wb") as f:
    shutil.copyfileobj(response.raw, f)

# Extract the gzip file
with gzip.open(output_gz_file, "rb") as f_in:
    with open(output_csv_file, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

print(f"File downloaded and extracted to {output_csv_file}")

In [ ]:
'''
ID = 11 character station identification code
YEAR/MONTH/DAY = 8 character date in YYYYMMDD format (e.g. 19860529 = May 29, 1986)
ELEMENT = 4 character indicator of element type 
DATA VALUE = 5 character data value for ELEMENT 
M-FLAG = 1 character Measurement Flag 
Q-FLAG = 1 character Quality Flag 
S-FLAG = 1 character Source Flag 
OBS-TIME = 4-character time of observation in hour-minute format (i.e. 0700 =7:00 am)
'''
weather_df = pd.read_csv(output_csv_file,
                         header=None,
                         names=['ID', 'YEAR/MONTH/DAY',
                                'ELEMENT', 'DATA_VALUE',
                                'M_FLAG', 'Q_FLAG', 'S_FLAG',
                                'OBS_TIME'],
                         low_memory=False)

In [ ]:
# Convert YEAR/MONTH/DAY column to Date type
weather_df['DATE'] = pd.to_datetime(weather_df['YEAR/MONTH/DAY'], format='%Y%m%d')
weather_df.head()

### Add snowfall data to flight data

In [ ]:
# Filter daily snowfall records
snow_df = weather_df[weather_df['ELEMENT'] == 'SNOW'].copy()
snow_df.rename(columns={'DATA_VALUE': 'DAILY_SNOWFALL'}, inplace=True)

# Temporarily create a date-only column
flights_df['DATE_ONLY'] = flights_df['DATE'].dt.floor('d')

# Rename the DATE column in snow_df to avoid conflicts
snow_df.rename(columns={'DATE': 'SNOW_DATE'}, inplace=True)

# Merge on DATE_ONLY to match daily snowfall
flights_df = flights_df.merge(
    snow_df[['ID', 'SNOW_DATE', 'DAILY_SNOWFALL']],
    left_on=['origin_airport/NOAA_STATION_ID', 'DATE_ONLY'],
    right_on=['ID', 'SNOW_DATE'],
    how='left'
)

# Clean up columns no longer needed
flights_df.drop(columns=['DATE_ONLY', 'ID', 'SNOW_DATE'], inplace=True)

### Export Cleaned Data

In [ ]:
# Export cleaned data to Parquet file
flights_df.to_parquet("cleaned_flights.parquet", index=False)